In [2]:
from importlib import reload
import folium
import pandas as pd
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
import src.localization as localization
reload(localization)
from src.localization import read_data
from src.localization import get_distance_in_meters

In [3]:
positions, df = read_data('resources/data/LocTreino_Equipe_2.csv')
bts_positions, df_btss = read_data('resources/data/dados_BTSs.csv')
df

,pontoId,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
0,160,-8.076634,-34.899639,122.556667,111.290000,126.590000,130.623333,132.656667,140.056667,2,1.0,1.0,1,2,2
1,2481,-8.065599,-34.887295,119.240000,131.965000,130.415000,125.090000,125.090000,132.915000,1,2.0,4.0,2,0,1
2,1986,-8.068570,-34.896713,105.690000,121.890000,118.840000,118.440000,119.290000,106.490000,0,1.0,2.0,1,1,0
3,923,-8.072540,-34.892147,105.640000,128.790000,111.540000,105.440000,109.640000,122.540000,0,0.0,3.0,0,1,1
4,1429,-8.070015,-34.890034,104.165000,140.240000,124.790000,120.840000,106.140000,128.490000,0,1.0,3.0,1,0,1
5,1719,-8.069263,-34.895710,96.356667,123.523333,120.056667,108.256667,110.223333,116.990000,0,1.0,2.0,1,1,0
6,2265,-8.066721,-34.896595,112.123333,108.923333,124.623333,113.823333,127.090000,87.823333,0,1.0,3.0,1,1,0
7,590,-8.074231,-34.900223,124.290000,103.990000,96.490000,116.256667,134.056667,139.290000,1,1.0,1.0,1,2,2
8,967,-8.072442,-34.893211,98.423333,116.856667,114.590000,99.656667,109.523333,115.656667,0,0.0,3.0,0,1,1
9,229,-8.076364,-34.899738,125.556667,111.156667,136.056667,134.123333,141.123333,142.123333,2,1.0,1.0,1,2,2


Using Time Advance
==

In [11]:
# Calculate map center
center_lat = np.mean([lat for lat in bts_positions[:, 0]])
center_lon = np.mean([lon for lon in bts_positions[:, 1]])
original_pts_map = folium.Map(location=[center_lat, center_lon], zoom_start=15)
# Plot points
for location in positions:
    marker = folium.CircleMarker(location=location,
                                 radius=1,
                                 color='blue',
                                 fill_opacity=0.1)
    #marker.add_to(original_pts_map)

# Plot BTSs locations
#for location in bts_positions:
marker = folium.Marker(location=bts_positions[3])
marker.add_to(original_pts_map)

# Export map
original_pts_map.save("resources/maps/original_pts_map_timing_advance.html")

# Show map
original_pts_map

In [12]:
lat_bts = df_btss['lat'][0]
lon_bts = df_btss['lon'][0]
time_advance_expected = 0
print(lat_bts)
print(lon_bts)
distance = get_distance_in_meters(lat_bts,lon_bts,-8.06500,-34.900)
distance

-8.068361111
-34.892722221999996


884.4006554772664

In [13]:
from operator import itemgetter, attrgetter

instance = df.iloc[[1480]]
#In this Step, we will select the bts with the least pathloss, it means that the ms is probably near
#
bts1 = list(instance[['pathBTS1','taBTS1']].values)
bts2 = list(instance[['pathBTS2','taBTS2']].values)
bts3 = list(instance[['pathBTS3','taBTS3']].values)
bts4 = list(instance[['pathBTS4','taBTS4']].values)
bts5 = list(instance[['pathBTS5','taBTS5']].values)
bts6 = list(instance[['pathBTS6','taBTS6']].values)
#
#
bts_infos = [bts1,bts2,bts3,bts4,bts5,bts6]
#bts_infos.sort(key=lambda x: x[0])

lat_instance = instance['lat'].values[0]
lon_instance = instance['lon'].values[0]
print(lat_instance)
print(lon_instance)

#some_list = [(1,2),(2),(5),(3),(4),(7),(0)]
#some_list.sort()
#some_list
#bts_infos
bts_by_pathloss = [
       [bts1[0][0],bts1[0][1],0],
       [bts2[0][0],bts2[0][1],1],
       [bts3[0][0],bts3[0][1],2],
       [bts4[0][0],bts4[0][1],3],
       [bts5[0][0],bts5[0][1],4],
       [bts6[0][0],bts6[0][1],5]]
bts_sorted = sorted(bts_by_pathloss,key=itemgetter(0))
bts_sorted

-8.07012
-34.900906


[[117.64, 1.0, 2],
 [121.09, 1.0, 3],
 [123.24, 1.0, 5],
 [125.24, 1.0, 0],
 [127.79, 1.0, 1],
 [131.59, 2.0, 4]]

In [14]:
print(distance)
time_advance_expected = (distance/550) - 1
round(time_advance_expected)
print(bts_positions)
print(bts_sorted[0][1])
print(bts_sorted[0][2])
print(bts_sorted[0][0])

884.4006554772664
[[ -8.06836111 -34.89272222]
 [ -8.07591667 -34.89461111]
 [ -8.07636111 -34.908     ]
 [ -8.07591667 -34.89461111]
 [ -8.066      -34.88944444]
 [ -8.06458333 -34.89458333]]
1.0
2
117.64


In [15]:
#Estimate the radius that
#Ta Seboso, mas depois ajeitamos
time_advance = bts_sorted[1][1]
min_radius = (time_advance) * 550
max_radius = (time_advance + 1) * 550



bts_index = bts_sorted[1][2] #Select the nearest bts index
print(bts_index)
lat_bts = df_btss['lat'][bts_index]
lon_bts = df_btss['lon'][bts_index]

print(lat_bts)
print(lon_bts)
distance = get_distance_in_meters(lat_bts,lon_bts,-8.06500,-34.900)

if(distance <=max_radius and distance > min_radius):
    print('A distância é de {} metros Está dentro do range previsto'.format(distance))
#marker = folium.Circle(location=(lat_bts,lon_bts),color='blue',weight=1,radius=distance)
#marker.add_to(original_pts_map)

icon_p = folium.Icon(color='black',icon_color='red')
marker = folium.Circle(location=(lat_bts,lon_bts),color='green',weight=1,radius=min_radius)
marker.add_to(original_pts_map)
marker = folium.Circle(location=(lat_bts,lon_bts),color='red',weight=1,radius=max_radius)
marker.add_to(original_pts_map)
marker = folium.Marker(location=(lat_instance,lon_instance),popup='teste',tooltip='celular',icon=icon_p)
marker.add_to(original_pts_map)
print(min_radius)
print(max_radius)

3
-8.075916667000001
-34.8946111116
550.0
1100.0


In [16]:
original_pts_map

Saving Output File
==

In [9]:
df_test_output = pd.read_csv('resources/data/LocTreino_Equipe_3.csv')

In [10]:
df_test_output.head()

,pontoId,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
0,328,-8.075851,-34.899895,122.723333,112.056667,121.790000,132.423333,140.823333,138.123333,2,1,1,1,2.0,2
1,1601,-8.069568,-34.890842,108.723333,132.456667,124.256667,114.790000,102.923333,106.056667,0,1,3,1,0.0,1
2,938,-8.072496,-34.896740,104.990000,104.178889,113.690000,93.290000,121.423333,132.123333,1,0,2,0,1.0,1
3,710,-8.073412,-34.900562,128.940000,117.690000,121.340000,125.990000,142.340000,134.540000,1,1,1,1,2.0,2
4,739,-8.073216,-34.896381,93.590000,113.040000,91.940000,104.290000,119.440000,127.540000,1,0,2,0,2.0,1


In [11]:
from numpy.random import randn

dic = {'pontoId':df_test_output['pontoId'],
       'lat':df_test_output['lat'],
       'lon':df_test_output['lon'],
       'lat_pred':predicted_latitudes,
       'lon_pred':predicted_longitudes,
       'erro_loc':error_in_meters}
df_results = pd.DataFrame(dic,columns=dic.keys())
df_results

NameError: name 'predicted_latitudes' is not defined

In [ ]:

df_test_output.to_csv('resources/data/Resultados.csv')